<a href="https://colab.research.google.com/github/kimwoonggon/publicservant_AI/blob/master/%EC%8B%AC%ED%99%943_GPT2_%EA%B5%AC%ED%98%84%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT2
오늘은 GPT2를 텐서플로우 2.0을 활용하여 직접 훈련시켜보고자 합니다.  
순서는 <b>1)sentencepiece를 활용한 데이터 전처리 -> 2)임베딩 -> 3)멀티헤드어텐션 -> 4)피드 포워드 네트워크 -> 5)디코더 -> 6)아웃풋 -> 7)추론</b> 으로 구성됩니다.

## 개요
GPT2를 레이어를 직접 처음부터 쌓아보고자 합니다.  


트랜스포머에 사용했던 데이터인 <b>포르투갈어->영어</b> 데이터를 활용하여 GPT2를 직접 학습시킵니다.  
이번 튜토리얼에서는 GPT2를 일반적인 데이터로 훈련시키기에는 너무 많은 시간이 소요되기 때문에 가벼운 데이터셋으로 하려고 해요.  
GPT2는 트랜스포머만 잘 이해했으면 어려울 것이 없습니다.  
인코더-디코더로 이루어진 트랜스포머와 다른 점은 단지 디코더로만 이루어졌을 뿐이기 때문입니다. 참고로 트랜스포머 모델 중 유명한 모델인 버트는 인코더로만 이루어져 있습니다.  
포르투갈어를 영어로 바꾸는 문제를 예시로 들면, 기존 트랜스포머에서는 포르투갈어가 인코더로 들어가고, 인코딩된 결과가 디코더와 상호작용 하면서 영어로 아웃풋을 내는 번역을 했었습니다.   
그리고 훈련 시킬 때에는 영어도 디코더 인풋에 들어갔었지요. 
![Imgur](https://i.imgur.com/mZRCi1o.png)  
인코더 인풋 : os meus vizinhos ouviram sobre esta ideia.  
디코더 인풋 : and my neighboring homes heard about this idea .  
번역 결과 : and my neighboring homes heard about this idea .
  
하지만 **GPT2는 (포르투갈어, 구분자, 영어)가 한 문장에** 들어가게 됩니다.  
예를 들면, 기존 트랜스포머에서는 os meus vizinhos ouviram sobre esta ideia. 라는 포르투갈어가  인코더로 들어가고, and my neighboring homes heard about this idea . 라는 영어가 디코더로 들어가서 서로 인코더와 디코더가 상호작용 해서 모델이 훈련이 되었습니다.  

  
그런데 GPT2에서는  
**인풋: os meus vizinhos ouviram sobre esta ideia.<구분자>and my neighboring homes heard about this idea .  <끝맺음>**
두 문장이 한번에 들어가게 됩니다.  왜냐하면 인코더가 없으니 디코더에 모든 정보를 넣어 주어야 하기 때문입니다.  
훈련은 앞단어를 활용해서 뒷단어를 예측하는 문제로 아주 간단합니다.  
![Imgur](https://i.imgur.com/RKl4lUt.png)  
인풋은 앞단어로 뒷단어를 추론하는 방식을 취합니다.  


인풋 : os meus vizinhos ouviram sobre esta ideia.<구분자>and my neighboring homes heard about this idea .   

아웃풋 : meus vizinhos ouviram sobre esta ideia.<구분자>and my neighboring homes heard about this idea . <끝맺음>  


# 데이터 전처리
텐서플로우에서 제공하는 포르투갈어를 영어로 번역하는 데이터를 불러옵니다.  
sentencepiece로 단어에 인덱스를 부여하는 기능을 만들 것이고, 데이터를 텐서플로우 dataset으로 만들어 줄 것입니다.  

In [1]:
# 포르투갈어 -> 영어 데이터 가져오기
!pip install -q tf-nightly
import tensorflow_datasets as tfds

     |████████████████████████████████| 525.1MB 25kB/s 
     |████████████████████████████████| 2.9MB 39.8MB/s 
     |████████████████████████████████| 460kB 10.1MB/s 


In [0]:
import tensorflow as tf
import numpy as np
import math

In [3]:
# 포르투갈어 -> 영어 데이터 불러오기

examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True,
                               as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteBQBHEJ/ted_hrlr_translate-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteBQBHEJ/ted_hrlr_translate-validation.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteBQBHEJ/ted_hrlr_translate-test.tfrecord


Dataset ted_hrlr_translate downloaded and prepared to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0. Subsequent calls will reuse this data.


In [0]:
train_examples, val_examples, test_examples = examples['train'], examples['validation'], examples['test']

In [5]:
# tensorflow dataset iterator로 되어 있다
examples

{'test': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.string)>,
 'train': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.string)>,
 'validation': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.string)>}

In [0]:
train_examples, val_examples, test_examples = examples['train'], examples['validation'], examples['test']

sentencepiece를 활용하여 토큰 처리를 해주기 위해 리스트로 바꿔 준다. 이후 txt 파일로 만들 것이다.

In [0]:
# pt : 포르투갈어
# eng : 영어
# pt_val : 포르투갈어(검증 데이터 셋)
# eng_val : 영어(검증 데이터 셋)
pt = [x.numpy().decode() for x,y in train_examples]
eng = [y.numpy().decode() for x,y in train_examples]
pt_val = [x.numpy().decode() for x,y in val_examples]
eng_val = [y.numpy().decode() for x,y in val_examples]

In [0]:
# 데이터들을 txt 파일로 변경해준다
train_data = pt + eng + pt_val + eng_val
f = open("train.txt", 'w')
for i in range(len(train_data)):
  data =train_data[i] + "\n"
  f.write(data)
f.close()

In [9]:
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 2.8MB/s 


In [10]:
# sentencepiece를 활용하기
templates = '--input={} --pad_id={} --bos_id={} --eos_id={} --unk_id={} --model_prefix={} --vocab_size={} --character_coverage={} --model_type={}'


train_input_file = 'train.txt' # train.txt에서 문장 불러 오기
pad_id = 0
vocab_size = 40000 # 최대 단어 수는 40000으로 한다
prefix = "gpt2_spm" # sentencepiece가 gpt2_spm.model gpt2_spm.vocab으로 저장된다.
unk_id = 1 # 모르는 단어 [UNK]의 인덱스
bos_id = 2 # Beginning of sentence, 문장의 시작을 알려주는 인덱스
eos_id = 3 # End of sentecnce, 문장의 끝을 알려주는 인덱스
character_coverage = 1.0 # 한국어는 0.9정도가 좋음.(단어 하나가 함의하는게 클수록 작게 준다)
model_type = 'bpe' # 토크나이징에 bpe 기법을 사용 예정

cmd = templates.format(train_input_file,
                       pad_id,
                       bos_id,
                       eos_id,
                       unk_id,
                       prefix,
                       vocab_size,
                       character_coverage,
                       model_type)
print(cmd)
import sentencepiece as spm

spm.SentencePieceTrainer.Train(cmd) # sentencepiece 모델을 훈련시킨다

sp = spm.SentencePieceProcessor()
sp.Load("gpt2_spm.model") # 훈련한 모델을 불러온다



--input=train.txt --pad_id=0 --bos_id=2 --eos_id=3 --unk_id=1 --model_prefix=gpt2_spm --vocab_size=40000 --character_coverage=1.0 --model_type=bpe


True

In [0]:
# vocab 파일을 불러와서 단어 사전을 만든다
with open("gpt2_spm.vocab", encoding='utf-8') as f:
  Vo = [doc.strip().split("\t") for doc in f]
word2idx = {w[0] : i for i, w in enumerate(Vo)}

In [12]:
print(word2idx)

{'<pad>': 0, '<unk>': 1, '<s>': 2, '</s>': 3, '▁a': 4, '▁t': 5, '▁,': 6, '▁.': 7, '▁e': 8, '▁d': 9, 'er': 10, '▁i': 11, '▁s': 12, '▁th': 13, '▁p': 14, '▁c': 15, '▁o': 16, '▁m': 17, 'es': 18, 'os': 19, 'en': 20, 'ar': 21, 'as': 22, '▁n': 23, '▁w': 24, 'in': 25, 'or': 26, 'qu': 27, 'an': 28, '▁f': 29, 're': 30, 'at': 31, '▁the': 32, 'is': 33, 'on': 34, "▁'": 35, '▁b': 36, 'ou': 37, 'al': 38, '▁de': 39, '▁qu': 40, 'om': 41, '▁h': 42, 'nd': 43, '▁u': 44, 'em': 45, 'it': 46, '▁l': 47, 'am': 48, 'ent': 49, 'st': 50, 'ad': 51, '▁to': 52, 'ic': 53, 'ão': 54, '▁in': 55, '▁que': 56, 'ing': 57, '▁and': 58, '▁v': 59, "▁''": 60, 'ed': 61, '▁g': 62, 'el': 63, '▁of': 64, '▁com': 65, 'ro': 66, 'id': 67, '▁um': 68, 'ra': 69, 'ri': 70, 'le': 71, '▁y': 72, 'ut': 73, '▁that': 74, '▁re': 75, '▁do': 76, 'ec': 77, '▁est': 78, 'il': 79, '▁is': 80, '▁no': 81, 've': 82, 'et': 83, '▁it': 84, '▁`': 85, '▁``': 86, '▁par': 87, 'ol': 88, '▁é': 89, '▁you': 90, '▁wh': 91, 'ia': 92, 'im': 93, 'ion': 94, '▁we': 95, '▁se

In [13]:
print("총 단어 수", sp.get_piece_size())
# 토큰화
print("포르투갈어", pt[0])
print("영어", eng[0])
print("포르투갈어 인덱스 부여", sp.EncodeAsIds(pt[0]))
print("인덱스를 토큰으로(포르투갈어)", sp.EncodeAsPieces(pt[0]))
print("인덱스를 토큰으로(영어)", sp.EncodeAsPieces(eng[0]))
#토큰 -> 문자열
print("토큰을 원래 문장으로", sp.DecodePieces(sp.EncodeAsPieces(eng[0])))
print("시작단어",sp.id_to_piece(2), "끝단어",sp.id_to_piece(3))

총 단어 수 40000
포르투갈어 e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade .
영어 and when you improve searchability , you actually take away the one advantage of print , which is serendipity .
포르투갈어 인덱스 부여 [8, 347, 883, 168, 4, 3646, 6, 13430, 4, 2292, 9068, 138, 7232, 6, 56, 89, 4, 32173, 260, 7]
인덱스를 토큰으로(포르투갈어) ['▁e', '▁quando', '▁melhor', 'amos', '▁a', '▁procura', '▁,', '▁tiramos', '▁a', '▁única', '▁vantagem', '▁da', '▁impressão', '▁,', '▁que', '▁é', '▁a', '▁serendip', 'idade', '▁.']
인덱스를 토큰으로(영어) ['▁and', '▁when', '▁you', '▁improve', '▁search', 'ability', '▁,', '▁you', '▁actually', '▁take', '▁away', '▁the', '▁one', '▁advantage', '▁of', '▁print', '▁,', '▁which', '▁is', '▁serendipity', '▁.']
토큰을 원래 문장으로 and when you improve searchability , you actually take away the one advantage of print , which is serendipity .
시작단어 <s> 끝단어 </s>


텐서플로우에서 활용할 수 있도록 train 데이터를 만들어 보겠다.  
포르투갈어 <구분자> 영어 <끝> 의 구성을 취할 것이다.

In [0]:
def encode(lang1, lang2):
  # 포르투갈어 + <영어 시작> + 영어 + <영어 끝>
  lang = sp.EncodeAsIds(lang1) + [sp.piece_to_id("<s>")] + sp.EncodeAsIds(lang2) + [sp.piece_to_id("</s>")]# 포르투갈 어를 인코딩 할 때 시작 단어를 의미하는 숫자와, 끝 단어를 의미하는 숫자가 붙음
  # 텐서플로우에서 활용할 수 있는 텐서로 바꿔준다
  lang = tf.convert_to_tensor(lang)
  # 텐서의 모양을 정해줘야 함, 문장의 길이는 변하므로 [None]으로
  lang.set_shape([None])
  return lang

In [15]:
print(encode(pt[1], eng[1]))

tf.Tensor(
[  174     8    96   981  9414  5329 12064   121     2   180   190   306
    84   439  7192   121     3], shape=(17,), dtype=int32)


In [0]:
# texts = 트레인 데이터 셋
# texts_val = valid 데이터 셋
texts = [encode(x,y) for x,y in zip(pt, eng)]
texts_val = [encode(x,y) for x,y in zip(pt_val, eng_val)]

In [0]:
MAX_LENGTH = 100
BUFFER_SIZE = 60000
BATCH_SIZE = 64

# 만약 문장이 인코딩 되었을 때, 인코딩 된 길이가 100을 초과하면 데이터에서 배제하고자 하는 필터 함수임
def filter_max_length(x, max_length=MAX_LENGTH):
  return tf.size(x) <= max_length

In [0]:
# Train 데이터(texts)를 generator를 활용하여 텐서플로우 dataset을 만듦
def generator():
    for el in texts:
        yield el
dataset = tf.data.Dataset.from_generator(generator,
                                           output_types= tf.int64)
dataset = dataset.filter(filter_max_length) # 길이가 100 초과하면 날리기
dataset = dataset.cache() # cache를 활용해서 데이터를 로드할 때 빠른 처리를 기대해 봄
dataset = dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=[None]) # padded_batch 불러오기
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE) # 데이터가 훈련되는 사이에, 데이터를 미리 준비해 놓음

In [0]:
# valid 데이터 또한 텐서플로우 dataset으로 만들기
def val_generator():
    for el in texts_val:
        yield el
val_dataset = tf.data.Dataset.from_generator(val_generator,
                                           output_types= tf.int64)
val_dataset = val_dataset.filter(filter_max_length).padded_batch(BATCH_SIZE, padded_shapes=[None])

# GPT2 모델 쌓기

## 임베딩
임베딩 레이어를 통해 단어 하나하나를 고차원으로 만들어 줌  
임베딩은 포지션 임베딩과 언어 임베딩으로 이루어져 있습니다.  
**인풋 -> [batch_size, 문장 길이]**  
**아웃풋 -> [batch_size, 문장 길이, 임베딩 차원]**  
![Imgur](https://i.imgur.com/KVkleNI.png)  
같은 단어가 임베딩 되었더라도, 포지션 임베딩이 합쳐서 문장 내부의 단어의 순서에 따라 임베딩 값은 달라지게 됩니다.
![Imgur](https://i.imgur.com/sSnVRpY.png)  

In [0]:
# 단어 임베딩 정의
class EmbeddingLayer(tf.keras.layers.Layer):

  def __init__(self, vocab_size, embedding_size, initializer=None, stddev=0.02, mean=0.0):
    super().__init__(self)
    self.vocab_size = vocab_size # 훈련할 데이터의 vocab size
    self.embedding_size = embedding_size # 단어가 고차원으로 임베딩되는 차원
    self.stddev = stddev # 초기 임베딩 계수들의 표준편차
    self.mean = mean # 초기 임베딩 계수들의 평균
    self.initializer = initializer
    if self.initializer is None:
      self.initializer = tf.random_normal_initializer(mean=self.mean,
                                                      stddev = self.stddev) # N~(0, 2^2)의 분포로 초기 계수를 정규화
  def build(self, input_shape):
     with tf.name_scope("embedding_weights"):
      self.embedding_weights = self.add_weight(
          "weights",
          shape=[self.vocab_size, self.embedding_size],
          dtype="float32",
          initializer=self.initializer) # [vocab_size, embedding_size]의 계수 정의
  
  def call(self, inputs, mode="embedding", scale=False):
    if mode == "embedding":
      return self.embedding(inputs, scale=scale) # embedding 모드이면, 인풋 단어를 임베딩 할 때 사용한다
    elif mode == "projection":
      return self.projection(inputs) # projection 모드이면, 최종 출력 값에 임베딩 계수들을 다시 곱해준다
    else:
      raise ValueError("mode {} is not valid.".format(mode))
  
  def embedding(self, inputs, scale=False):
    with tf.name_scope("embedding"):
      mask = tf.cast(tf.not_equal(inputs, 0), tf.float32) # 마스크를 만들어서 패딩인 부분을 0으로 마스크
      inputs = tf.cast(inputs, tf.int32) # 인풋 값을 텐서로 바꿔줌, input_shape -> [batch_size, length]

      embeddings = tf.nn.embedding_lookup(self.embedding_weights, inputs) # 임베딩 룩업 테이블을 곱해서 단어를 임베딩
      # embeddings_shape -> [batch_size, length, embedding_size]
      # mask size is converted to [batch_size, length, 1]
      embeddings *= tf.expand_dims(mask, -1)

      if scale:
        embeddings *= self.embedding_size ** 0.5

      return embeddings
  
  # GPT2 최종 출력 값에, 임베딩 벡터를 다시 곱해줌
  def projection(self, inputs):
    with tf.name_scope("output_layer"):
      batch_size = tf.shape(inputs)[0]
      seq_len = tf.shape(inputs)[1]
      # 출력 값의 input shape [batch_size, seq_len, d_model] 이다
      # seq_len : 문장 길이, d_model = 단어 embedding 차원
      h_flat = tf.reshape(inputs, [-1, self.embedding_size])
      # h_flat shape = [batch_size*seq_len, d_model]
      logits = tf.matmul(h_flat, self.embedding_weights, transpose_b=True)
      # logits shape = [batch_size*seq_len, vocab_size]
      return tf.reshape(logits, [batch_size, seq_len, self.vocab_size])
      # final output shape = [batch_size, seq_len, vocab_size]

# 포지셔널 임베딩 정의
class PositionEmbeddingLayer(tf.keras.layers.Layer):

  def __init__(self, position_seq, pos_embedding_size, trainable=True, stddev=0.02, mean=0.0):
    tf.keras.layers.Layer.__init__(self, name="PositionEmbeddingLayer")
    
    self.position_seq = position_seq # 보통 적당히 큰 숫자(10000) 정도로 입력한다음 문장의 길이에 맞게 자름
    self.hidden_size = pos_embedding_size
    self.trainable = trainable
    self.stddev = stddev
    self.mean = mean

    if trainable:
      self.position_embedding = EmbeddingLayer(self.position_seq, self.hidden_size, stddev=self.stddev, mean=self.mean)

  def call(self, inputs, start=1):
    with tf.name_scope("pos_embedding"):
      if self.trainable:
        batch_size = tf.shape(inputs)[0]
        batch_seq = tf.shape(inputs)[1]

        positions = tf.reshape(tf.tile(tf.range(start, batch_seq + start), [batch_size]), [batch_size, batch_seq])
        # positions는 직접 해보면 좋음
        positions = tf.cast(positions, tf.int32)
        position_mask = tf.cast(tf.not_equal(inputs, 0), tf.int32)
        positions *= position_mask

        return self.position_embedding(positions)

      else:
        return self.get_position_sinusoid(self.position_seq, self.hidden_size) # 트랜스포머에서 한 것과 유사함. 생략해도 좋음

  @staticmethod
  def get_position_sinusoid(seq_len, hidden_size, min_timescale=1.0, max_timescale=1.0e4):
    position = tf.cast(tf.range(seq_len), tf.float32)
    num_timescales = hidden_size // 2
    log_timescale_increment = (math.log(float(max_timescale) / float(min_timescale)) / (tf.cast(num_timescales, tf.float32) - 1))
    inv_timescales = min_timescale * tf.exp(tf.cast(tf.range(num_timescales), tf.float32) * -log_timescale_increment)
    scaled_time = tf.expand_dims(position, 1) * tf.expand_dims(inv_timescales, 0)
    signal = tf.concat([tf.sin(scaled_time), tf.cos(scaled_time)], axis=1)
    return signal

In [21]:
embed_test = EmbeddingLayer(10000, 512)
embed_test(np.random.uniform(size=(1,40)).astype('float32'))

<tf.Tensor: shape=(1, 40, 512), dtype=float32, numpy=
array([[[ 0.02587487, -0.01541876, -0.0180132 , ...,  0.00371212,
          0.01483497,  0.00364437],
        [ 0.02587487, -0.01541876, -0.0180132 , ...,  0.00371212,
          0.01483497,  0.00364437],
        [ 0.02587487, -0.01541876, -0.0180132 , ...,  0.00371212,
          0.01483497,  0.00364437],
        ...,
        [ 0.02587487, -0.01541876, -0.0180132 , ...,  0.00371212,
          0.01483497,  0.00364437],
        [ 0.02587487, -0.01541876, -0.0180132 , ...,  0.00371212,
          0.01483497,  0.00364437],
        [ 0.02587487, -0.01541876, -0.0180132 , ...,  0.00371212,
          0.01483497,  0.00364437]]], dtype=float32)>

In [22]:
test = np.random.uniform(size=(1,40)).astype('float32')
pos_embed_test = EmbeddingLayer(test.shape[-1], 512)
pos_embed_test(test)

<tf.Tensor: shape=(1, 40, 512), dtype=float32, numpy=
array([[[ 0.0144738 ,  0.02034662, -0.03761116, ..., -0.00787539,
         -0.03269539,  0.01726196],
        [ 0.0144738 ,  0.02034662, -0.03761116, ..., -0.00787539,
         -0.03269539,  0.01726196],
        [ 0.0144738 ,  0.02034662, -0.03761116, ..., -0.00787539,
         -0.03269539,  0.01726196],
        ...,
        [ 0.0144738 ,  0.02034662, -0.03761116, ..., -0.00787539,
         -0.03269539,  0.01726196],
        [ 0.0144738 ,  0.02034662, -0.03761116, ..., -0.00787539,
         -0.03269539,  0.01726196],
        [ 0.0144738 ,  0.02034662, -0.03761116, ..., -0.00787539,
         -0.03269539,  0.01726196]]], dtype=float32)>

## 멀티헤드 어텐션  
심화 2에서 봤던 트랜스포머의 멀티 헤드 어텐션과 거의 비슷함  
http://jalammar.github.io/illustrated-gpt2/ 에 아주 설명이 잘 되어 있음  


In [0]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, att_dropout=0.1, residual_dropout=0.1, scale=True):
    super().__init__(self, name="MultiHeadAttention")
    self.num_heads = num_heads # 병렬로 나눠서 처리할 숫자
    self.d_model = d_model # 임베딩 차원
    self.att_dropout = att_dropout # 어텐션 드롭아웃
    self.residual_dropout = residual_dropout # 레지듀얼 드롭아웃
    self.scale = scale # 스케일링 할지 말지 (Query * Value의 값을 SQRT(임베딩차원) 으로 나눔)

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.c_attn = tf.keras.layers.Dense(self.d_model * 3) # 출력 차원을 3배로 한 다음에 q, k, v로 나눔 
    self.c_proj = tf.keras.layers.Dense(self.d_model)

  def split_heads(self, x):
    batch_size = tf.shape(x)[0]
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0,2,1,3])

  def multihead_attention(self, q, k, v, training, mask=None):
    # depth : d_model // seq_len
    # q_shape : [batch_size, num_heads, seq_q, depth]
    # k_shape : [batch_size, num_heads, seq_k, depth]
    # v_shape : [batch_size, num_heads, seq_v, depth]

    matmul_qk = tf.matmul(q, k, transpose_b=True) # shape -> [batch_size, num_heads, seq_q, seq_k]
    if self.scale:
      dk = tf.cast(tf.shape(k)[-1], tf.float32)
      matmul_qk = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
      matmul_qk += (mask * -1e9) # 마스크를 곱해준다

    attention_weights = tf.nn.softmax(matmul_qk, axis=-1) # shape -> [batch_size, num_heads, seq_q, seq_k]

    if training:
      attention_weights = tf.nn.dropout(attention_weights, rate=self.att_dropout, name="attn_dropout")
    output = tf.matmul(attention_weights, v) # output shape -> [batch_size, num_heads, seq_q, depth]

    return output, attention_weights

  def merge_heads(self, x):
    batch_size = tf.shape(x)[0]
    # x shape -> [batch_size, num_heads, seq_q, depth]
    x = tf.transpose(x, perm=[0,2,1,3])
    # x shape -> [batch_size, seq_q, num_heads, depth]
    merged = tf.reshape(x, (batch_size, -1, self.d_model))
    # merged shape -> [batch_size, seq_q, d_model]
    return merged

  def call(self, x, mask=None, past_layer=None, training=True):
    # input x shape -> [batch_size, seq_len, d_model]
    x = self.c_attn(x)
    # x shape -> [batch_size, seq_len, d_model * 3]
    query, key, value = tf.split(x, 3, axis=2)
    # query_shape -> [batch_size, seq_len, d_model]
    # key_shape -> [batch_size, seq_len, d_model]
    # value_shape -> [batch_size, seq_len, d_model]

    query = self.split_heads(query)
    key = self.split_heads(key)
    value = self.split_heads(value)

    # query_shape -> [batch_size, num_heads, seq_len, d_model // num_heads]
    # key_shape -> [batch_size, num_heads, seq_len, d_model // num_heads]
    # value_shape -> [batch_size, num_heads, seq_len, d_model // num_heads]

    # past -> 과거 자료 활용
    # 예를 들어 "I have my chair repaired to study NLP"라는 문장이 있다고 하자
    # 마지막 단어를 예측할 때 NLP라는 단어를 예측한다고 하면
    # I have my chair repaired to study라는 문장을 GPT에 넣으면 NLP를 예측하게 된다.
    # 그런데 i have my chair repaired to 를 활용해서 study 라는 단어도 예측했을 것이다.
    # 따라서 i have my chair repaired to 를 GPT에 넣었을 때의 출력을 재활용하면
    # study라는 단어만 신경망을 거치면 되므로 예측 속도가 상당히 빨라지게 된다.

    if past_layer is not None:
      # past_layer의 shape은 [batch_size, 2, num_heads, seq_past_len, d_model // num_heads]
      past_key, past_value = tf.unstack(past_layer, axis=1)
      # past_key_shape = [batch_size, num_heads, seq_past_len, d_model // num_heads]
      # past_value_shape = [batch_size, num_heads, seq_past_len, d_model // num_heads]
      key = tf.concat([past_key, key], axis=-2)
      
      value = tf.concat([past_value, value], axis=-2)
      # key_value_shape = [batch_size, num_heads, seq_past_len + seq_len, d_model // num_heads]
      # value_value_shape = [batch_size, num_heads, seq_past_len + seq_len, d_model // num_heads]
    present = tf.stack([key, value], axis=1)
    # present shape -> [batch_size, 2, num_heads, seq_past_len + seq_len, d_model // num_heads]
    scaled_attention, attention_weights = self.multihead_attention(query, key, value, training, mask)
    #scaled_attention shape -> [batch_size, num_heads, seq_len, d_model // num_heads]
    #attention_weights_shape -> [batch_size, num_heads, seq_len, seq_past_len + seq_len, d_model // num_heads]
    concat_attention = self.merge_heads(scaled_attention)
    # concat_attention shape -> [batch_size, seq_len, d_model]
    output = self.c_proj(concat_attention)

    if training:
      output = tf.nn.dropout(output, rate=self.residual_dropout, name="resid_dropout")
    # output shape -> [batch_size, seq_len, d_model]
    return output, present

In [24]:
test = np.random.uniform(size=(1,40,512))
test = test.astype('float32')
print(test.shape)
attn = MultiHeadAttention(512, 8)
attn_out, _ = attn(test)
print(attn_out)

(1, 40, 512)
tf.Tensor(
[[[ 0.36504117  0.2511495  -0.05450708 ... -0.20046467  0.83729035
   -0.03749627]
  [ 0.47126028  0.         -0.01367733 ... -0.21993867  0.8900477
   -0.07379715]
  [ 0.38910648  0.1985568   0.01552593 ... -0.20587088  0.83503234
   -0.04676378]
  ...
  [ 0.438052    0.25525603 -0.02727513 ... -0.21067962  0.8397159
   -0.03844841]
  [ 0.39441177  0.26700345 -0.07513139 ... -0.19191273  0.
   -0.        ]
  [ 0.4164883   0.2260664   0.0395083  ... -0.21171942  0.8438246
   -0.04674293]]], shape=(1, 40, 512), dtype=float32)


## 포인트와이즈 피드 포워드 네트워크
멀티헤드어텐션의 아웃풋에 Weight을 곱해주어 신경망을 풍성하게 만들어줌

In [0]:
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, hidden_size, dff, dropout_rate=0.1, activation=tf.nn.relu):
    super().__init__(self, name="FeedForward")
    self.hidden_size = hidden_size # d_model
    self.dff = dff # middle dimension
    self.activation = activation # relu 
    self.dropout_rate = dropout_rate

    self.dense_layer = tf.keras.layers.Dense(self.dff)
    self.output_dense_layer = tf.keras.layers.Dense(self.hidden_size)

  def call(self, x, training=False):
    # input x shape -> [batch_size, seq_len, d_model]
    output = self.dense_layer(x)
    # shape -> [batch_size, seq_len, dff]
    output = self.activation(output)
    output = self.output_dense_layer(output)
    # shape -> [batch_size, seq_len, d_model]
    if training:
      output = tf.nn.dropout(output, rate=self.dropout_rate, name="feed_forward_dropout")

    return output

In [26]:
feed = FeedForward(512, 2048)
feed(attn_out)

<tf.Tensor: shape=(1, 40, 512), dtype=float32, numpy=
array([[[-0.24385557, -0.11730938, -0.1030661 , ..., -0.15606995,
         -0.23390052, -0.02218778],
        [-0.22112842, -0.22300558, -0.13148043, ..., -0.20515999,
         -0.22803116, -0.19408432],
        [-0.19557488, -0.17256711, -0.03311463, ..., -0.25110683,
         -0.22609097, -0.08730908],
        ...,
        [-0.24143258, -0.07318281, -0.02359992, ..., -0.14443266,
         -0.14642154, -0.12162777],
        [-0.21650189, -0.18980947, -0.03953472, ..., -0.07640225,
         -0.20449889, -0.079516  ],
        [-0.12534687, -0.20129989, -0.09789936, ..., -0.16241832,
         -0.1143719 , -0.05918843]]], dtype=float32)>

##디코더 정의

In [0]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, i=0, dropout_rate = 0.1):
    super().__init__(self, name="Decode_{}".format(i))
    self.d_model = d_model
    self.num_heads = num_heads
    self.dff = dff
    self.dropout_rate = dropout_rate

    self.mha = MultiHeadAttention(self.d_model, self.num_heads, self.dropout_rate)
    self.feed_forward = FeedForward(self.d_model, self.dff, self.dropout_rate)
    self.layer_norm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layer_norm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

  def call(self, x, training, mask, past=None):
    # input x -> [batch_size, seq_len, d_model]
    out, present = self.mha(self.layer_norm1(x), mask=mask, past_layer=past, training=training)
    # out -> [batch_size, seq_len, d_model]
    # present shape -> [batch_size, 2, num_heads, seq_past_len + seq_len, d_model // num_heads]
    with tf.name_scope("residual_conn_1"):
      x = x + out
    
    out = self.feed_forward(self.layer_norm2(x), training=training)
    with tf.name_scope("residual_conn_2"):
      x = x + out
    return x, present

In [28]:
test = np.random.uniform(size=(1,40,512)).astype('float32')
decoder = DecoderLayer(512, 8, 2048)
decoder(test, training=False, mask=None)

(<tf.Tensor: shape=(1, 40, 512), dtype=float32, numpy=
 array([[[ 0.52177   ,  0.15361214,  0.03117996, ...,  1.3114436 ,
          -0.61305237,  0.9057687 ],
         [ 0.35080394,  0.5839031 ,  0.09310269, ...,  0.23250967,
          -0.17617369,  0.978644  ],
         [ 0.95298916,  1.3240825 , -0.24399471, ...,  0.7709878 ,
          -0.9760052 ,  1.2204044 ],
         ...,
         [ 0.8418818 ,  0.49278337, -0.1096493 , ...,  1.0561212 ,
          -0.19908595,  0.5026806 ],
         [ 1.1991091 ,  0.67384034,  0.8147787 , ...,  1.038263  ,
           0.77711487, -0.21453206],
         [ 0.81821245,  0.01035579, -0.03581214, ...,  1.6309131 ,
          -0.431553  ,  1.6193246 ]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 2, 8, 40, 64), dtype=float32, numpy=
 array([[[[[ 0.5295211 ,  0.35052192, -0.78193164, ...,  0.47537988,
            -0.588601  ,  0.32514873],
           [-0.78332126, -0.9649495 ,  0.51740015, ...,  1.2118709 ,
            -0.28428847,  0.853191  ],
           [

## 출력층 정의  
출력증을 임베딩과 묶을지, 아니면 별도로 둘지 정할 수 있음

In [0]:
class OutputLayer(tf.keras.layers.Layer):
  def __init__(self, output_dim, proj_weights=None, kernel_initializer=None):
    super().__init__(self, name="OutputLayer")
    self.proj_weights = proj_weights
    self.output_dim = output_dim
    self.layer_weights = None
    self.kernel_initializer = kernel_initializer
    self.layer_weights = tf.keras.layers.Dense(self.output_dim)
  def call(self, x):
    # input x shape -> [batch_size, seq_len, d_model]
    batch, sequence, d_model = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[-1]

    if self.proj_weights is None:
      
      out = self.layer_weights(x)
      # out shape -> [batch_size, seq_len, vocab_size]
    else:
      out = self.proj_weights(x)
      # out shape -> [batch_size, seq_len, vocab_size]
    out = tf.reshape(out, [batch, sequence, self.output_dim])
    return out

In [30]:
test = np.random.uniform(size=(1,40,512)).astype('float32')
decoder = DecoderLayer(512, 8, 2048)
decoder_out, _ = decoder(test, training=False, mask=None)
output_layer = OutputLayer(9999)
output_layer(decoder_out)

<tf.Tensor: shape=(1, 40, 9999), dtype=float32, numpy=
array([[[ 0.29190174, -0.04769421, -0.12912437, ..., -0.18792886,
          0.10832113, -0.20609939],
        [ 0.2645763 ,  0.31869304,  0.11690763, ...,  0.11321118,
          0.09791943,  0.04935339],
        [ 0.343476  ,  0.28772736,  0.11091917, ..., -0.07169732,
         -0.15756083, -0.07677233],
        ...,
        [ 0.3152371 ,  0.11472939,  0.2163721 , ...,  0.01603666,
          0.11224319, -0.27735448],
        [-0.0053536 ,  0.08156694, -0.15305944, ...,  0.16289812,
          0.20620155, -0.15929136],
        [ 0.4308443 ,  0.20190167,  0.5399052 , ...,  0.19001153,
          0.1455318 , -0.05255146]]], dtype=float32)>

In [0]:
# 유틸리티 함수 정의
def shape_as_list_2(x):
    return [int(i) for i in tf.shape(x)]


def gelu(x):
    with tf.name_scope("gelu"):
        cdf = 0.5 * (1.0 + tf.tanh(
            (np.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3)))))
        return x * cdf


def get_padding_mask(seq):
    with tf.name_scope("Padding_Mask"):
        seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

        # add extra dimensions to add the padding
        # to the attention logits.
        return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)


def attention_mask(size):
    """
    if size is 4 then it returns below matrix
       [[0., 1., 1., 1.],
        [0., 0., 1., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.]]
    """
    with tf.name_scope("attention_mask"):
        mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
        return mask  # (seq_len, seq_len)


def create_masks(inp):
    with tf.name_scope("att_masking"):
        att_mask = attention_mask(tf.shape(inp)[1])
        padding_mask = get_padding_mask(inp)
        mask = tf.maximum(padding_mask, att_mask)

        return mask

## GPT2 정의
임베딩, 디코더, 출력층을 한번에 정의함

In [0]:
class GPT2(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, max_seq_len, vocab_size,\
               rev_embedding_projection=True):
    super().__init__(self, name="GPT2_TOTAL")

    self.rev_embedding_projection = rev_embedding_projection # 임베딩과 출력증을 묶을지
    self.num_layers = num_layers # 디코더 층
    self.num_heads = num_heads # 멀티 헤드 어텐션 때 헤드 숫자
    self.dff = dff # 중간층 차원
    self.max_seq_len = max_seq_len # position embedding시 임베딩할 최대 넓이
    self.vocab_size = vocab_size # 단어의 수
    self.d_model = d_model # 단어가 임베딩 될 때 차원
    
    self.embedding = EmbeddingLayer(self.vocab_size, self.d_model) # 단어 임베딩
    self.pos_embedding = PositionEmbeddingLayer(self.max_seq_len, self.d_model) # 위치 임베딩
    self.decoder_layers = [DecoderLayer(self.d_model, self.num_heads, self.dff, i) for i in range(self.num_layers)] # 디코더를 num_layers 만큼 쌓음
    
    if not self.rev_embedding_projection: # 만약 임베딩과 출력증을 같이 묶지 않으면
      self.output_layer = OutputLayer(self.vocab_size)

    self.layer_norm = tf.keras.layers.LayerNormalization(epsilon=1e-6)

  def call(self, x, training=True, past=None):
    
    x = tf.cast(x, tf.int32) # [batch_size, seq_len]
    batch, sequence = tf.shape(x)[0], tf.shape(x)[1]
    
    if past is None:
      pasts = [None] * self.num_layers
    else:
      pasts = past
    
    assert len(pasts) == self.num_layers

    att_mask = create_masks(x) # 어텐션 마스크

    past_length = 1 if past is None else tf.shape(past)[-2]

    with tf.name_scope("embeddings"):
      embedded_x = self.embedding(x)
      hidden_states = embedded_x + self.pos_embedding(x, start=past_length) # hidden_states -> [batch_size, seq_len, d_model]
    
    presents = []
    for decoder_layer, past in zip(self.decoder_layers, pasts):
      hidden_states, present = decoder_layer(hidden_states, training, att_mask, past=past)
      # hidden_states -> [batch_size, seq_len, d_model]
      # present shape -> [batch_size, 2, num_heads, seq_past_len + seq_len, d_model // num_heads]
      presents.append(present)

    hidden_states = self.layer_norm(hidden_states)

    if self.rev_embedding_projection: # 만약 임베딩과 출력층을 결합한다면
      logits = self.embedding(hidden_states, mode="projection")
      # shape -> [batch_size, seq_len, vocab_size]
    else:
      logits = self.output_layer(hidden_states)
      # shape -> [batch_size, seq_len, vocab_size]
    return logits, presents



In [33]:
x = np.random.uniform(size=(1,40)).astype('float32')
print(x)
gpt2 = GPT2(12, 512, 8, 2048, 400, 50000)
a, b = gpt2(x)
print(a)

[[0.92652446 0.5976662  0.7446475  0.59121805 0.28852233 0.6728189
  0.20890617 0.35280606 0.8582742  0.744634   0.59327155 0.5983983
  0.8249317  0.7184342  0.32953176 0.78165066 0.14222144 0.49829805
  0.7625399  0.92798644 0.88012075 0.34390682 0.73369724 0.24327196
  0.46360973 0.5430751  0.18734585 0.3297529  0.9618437  0.41318586
  0.661259   0.9992953  0.7719487  0.20186128 0.3749113  0.15442254
  0.18000929 0.10596427 0.8641406  0.50835663]]
tf.Tensor(
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]], shape=(1, 40, 50000), dtype=float32)


In [34]:
tf.cast(x, tf.float32)

<tf.Tensor: shape=(1, 40), dtype=float32, numpy=
array([[0.92652446, 0.5976662 , 0.7446475 , 0.59121805, 0.28852233,
        0.6728189 , 0.20890617, 0.35280606, 0.8582742 , 0.744634  ,
        0.59327155, 0.5983983 , 0.8249317 , 0.7184342 , 0.32953176,
        0.78165066, 0.14222144, 0.49829805, 0.7625399 , 0.92798644,
        0.88012075, 0.34390682, 0.73369724, 0.24327196, 0.46360973,
        0.5430751 , 0.18734585, 0.3297529 , 0.9618437 , 0.41318586,
        0.661259  , 0.9992953 , 0.7719487 , 0.20186128, 0.3749113 ,
        0.15442254, 0.18000929, 0.10596427, 0.8641406 , 0.50835663]],
      dtype=float32)>

# GPT2 훈련 준비하기

## 하이퍼파라미터 설정

In [0]:
num_layers = 4 # 디코더 층
d_model = 256 # 단어 임베딩 차원
num_heads = 4 # 어텐션 크기
dff = 512 # 중간 차원
max_seq_len = 10000 # 위치 임베딩의 길이
vocab_size = 40000 # 어휘 숫자

## 옵티마이저 설정

In [0]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps
    
  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)
    
    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

## Loss 함수 & Accuracy 정의

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none') #from_logits=True로 하면 Dense 이후 softmax layer 값 출력

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0)) # 예를 들어서 실제 자료(0은 패딩)가 [1,2,3,4,5,0,0,0,0,0] 이라면 [0,0,0,0,0,1,1,1,1,1]로 바꿔 줌
                                                     # 이후 tf.math.logical_not을 활용해서 [True,True,True,True,True,False,False,False,False,False]으로 바꿔 줌
  loss_ = loss_object(real, pred) # loss_는 패딩을 고려하지 않은 loss 값

  mask = tf.cast(mask, dtype=loss_.dtype) # [True,True,True,True,True,False,False,False,False,False]를 [1,1,1,1,1,0,0,0,0,0] 으로 바꿔 줌
  loss_ *= mask # loss에 mask를 곱해서, 패딩인 부분은 0처리 해줌

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


# 패딩이 아닌 부분만을 가지고 정확도를 측정함
def get_padded_accuracy(labels, logits):
  with tf.name_scope("padded_accuracy"):
    weights = tf.cast(tf.not_equal(labels, 0), tf.float32)

    outputs = tf.cast(tf.argmax(logits, axis=-1), tf.int32)
    padded_labels = tf.cast(labels, tf.int32)

    nonpad_seq = tf.math.count_nonzero(weights, dtype=tf.dtypes.float32, )
    acc = tf.cast(tf.equal(outputs, padded_labels), tf.float32)

    accuracy = tf.reduce_sum(tf.cast(acc * weights, tf.float32)) / nonpad_seq
  
  return tf.cast(accuracy, tf.float32)

## 저장 공간 설정

In [0]:
# 저장할 체크포인트 지정
checkpoint_path = "./"

ckpt = tf.train.Checkpoint(gpt2=gpt2,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

## Train 함수 설정

In [0]:
gpt2 = GPT2(num_layers = num_layers, d_model=d_model, num_heads=num_heads, dff=dff, max_seq_len=max_seq_len, vocab_size=vocab_size)
train_step_signature = [tf.TensorSpec(shape=(None, None), dtype=tf.int64)]

# tf.function을 사용하면 그래프를 미리 컴파일 하기 때문에 속도가 상당히 빠름
# 같은 GPU여도 케라스에 비해서 체감상 7~8배 정도의 차이가 나는 것 같음
@tf.function(input_signature=train_step_signature)
def train_step(inputs, grad_clip=True, clip_value=2.5):
  inp = inputs[:, :-1]
  tar = inputs[:, 1:]
  
  with tf.GradientTape() as tape:
    predictions, _ = gpt2(inp, True)
    loss = loss_function(tar, predictions)

  gradients = tape.gradient(loss, gpt2.trainable_variables)  
  if grad_clip:
    gradients = [(tf.clip_by_value(grad, -clip_value, clip_value)) for grad in gradients]  
  optimizer.apply_gradients(zip(gradients, gpt2.trainable_variables))
  
  accuracy = get_padded_accuracy(tar, predictions)
  return loss, accuracy

# 모델 훈련 및 추론

## 모델 훈련

In [47]:
import time
# 20 에포크 훈련
for epoch in range(20):
  start = time.time()
  
  #train_loss.reset_states()
  #train_accuracy.reset_states()
  
  # input : 포루투갈어, tar : 영어
  for (batch, inp) in enumerate(dataset):
    loss, accuracy = train_step(inp)
    
    if batch % 50 == 0:
      print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
          epoch + 1, batch, loss, accuracy))
      
  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
    
  print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                loss, 
                                                accuracy))

  print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 10.6782 Accuracy 0.0000
Epoch 1 Batch 50 Loss 10.4638 Accuracy 0.0000
Epoch 1 Batch 100 Loss 10.0336 Accuracy 0.0622
Epoch 1 Batch 150 Loss 9.5644 Accuracy 0.0575
Epoch 1 Batch 200 Loss 8.9974 Accuracy 0.0607
Epoch 1 Batch 250 Loss 8.2888 Accuracy 0.0882
Epoch 1 Batch 300 Loss 7.6124 Accuracy 0.1133
Epoch 1 Batch 350 Loss 7.0304 Accuracy 0.1078
Epoch 1 Batch 400 Loss 6.7934 Accuracy 0.1247
Epoch 1 Batch 450 Loss 6.3527 Accuracy 0.1515
Epoch 1 Batch 500 Loss 6.3763 Accuracy 0.1507
Epoch 1 Batch 550 Loss 6.2209 Accuracy 0.1614
Epoch 1 Batch 600 Loss 6.0919 Accuracy 0.1762
Epoch 1 Batch 650 Loss 5.9941 Accuracy 0.1757
Epoch 1 Batch 700 Loss 5.8707 Accuracy 0.1843
Epoch 1 Batch 750 Loss 5.8646 Accuracy 0.1721
Epoch 1 Loss 5.8836 Accuracy 0.1981
Time taken for 1 epoch: 140.41279816627502 secs

Epoch 2 Batch 0 Loss 5.5627 Accuracy 0.2089
Epoch 2 Batch 50 Loss 5.5822 Accuracy 0.2027
Epoch 2 Batch 100 Loss 5.8121 Accuracy 0.1878
Epoch 2 Batch 150 Loss 5.6438 Accuracy 0.205

## 추론
기존 transformer와 다른 점은 추론 시 softmax 이후 가장 높은 확률 만을 정답으로 보지 않는다는 점입니다.  
예를 들어서 설명드리자면 기존 추론 방식은 마지막 softmax(logits)의 값이 [0.5, 0.2, 0.1, 0.1, 0.05, 0.05] 라면 0.5가 속해있는 인덱스인 0을 정답으로 취합니다.   
하지만 GPT2의 추론에서는 정답 softmax 값에서 상위 k개의 값을 랜덤으로 뽑아서 정답으로 활용하게 됩니다.  
사실 가장 큰 확률을 가진 인덱스가 정답인 것이 타당합니다. 그러나 다른 관점에서 보면 확률 리스트 **[0.5, 0.2, 0.1, 0.1, 0.05, 0.05]에서 보면 0.5가 아닌 부분도** **50% 만큼 존재하는 것이고**, 이 말은 즉슨 50%의 확률로 정답 인덱스가 0이 아닐 수도 있기에 가장 개연성이 높은 값 말고도 다른 값을 고려하겠다는 뜻입니다.  


**top_k_logits** 함수와 **top_p_logits** 함수가 활용됩니다.  
top_k_logits는 출력 값에서 상위 k개만큼만 남기고 나머지는 0으로 처리합니다.  
만약에 logits이 [0.5, 0.2, 0.1, 0.1, 0.05, 0.05]이고, k=3이면 결과는 [0.5, 0.2, 0.1, 0, 0, 0]가 됩니다. 

top_p_logits 는 임계 확률 p를 정하고 누적 확률분포를 구합니다. 그 전에는 확률을 큰 수부터 sort가 이루어집니다.  
이후 임계확률 p를 넘어서는 부분부터는 0으로 처리합니다.
p의 기준점을 0.7이라고 하겠습니다.  
만약에 input이 [0.1, 0.05, 0.05, 0.5, 0.2, 0.1] 라면 **sort**가 되어서 [0.5, 0.2, 0.1, 0.1, 0.05, 0.05] 가 되고, **누적확률 분포**는 [0.5, 0.7, 0.8, 0.9, 0.95, 1.0]이 되는데 0.7이 넘는 점부터는 0이가 곱해지게 됩니다.  
그래서 임계점이 넘는 부분은 0처리가 되어서 결과는 [0.5, 0.7, 0, 0, 0, 0]이 됩니다.  
헷갈리실 수 있는데 아래 코드에서 -1e10을 곱한 이유는 tf.random.categorical 함수를 적용하기 위해서입니다. 이 함수는 softmax 함수가 적용되지 않은 logits를 인풋으로 받습니다.

Temperature 변수는 1을 기준으로 1보다 낮으면 추론 시 텍스트가 생성될 때 높은 확률의 단어 위주로 추론하게 되고,   
Temperature 변수가 1보다 크면 확률이 낮은 단어도 고려하게 됩니다.  
코드를 보시면  
<b>logits = logits[:, -1, :] / tf.cast(temperature, tf.float32)</b>  
logits을 temperature로 나눠주게 되는데, softmax = softmax(logits)일 때,     

만약 softmax 결과가 [0.8, 0.1, 0.1] 이라면 <b>temperature가 1보다 크면</b> softmax의 결과가 [0.75, 0.125, 0.125]와 같이 되어서 <b>낮은 확률의 값이 올라가서 top_k와 top_p 안에 들어갈 확률이 높아집니다.</b>  
  
반대로 <b>temperature가 1보다 작으면</b> 반대로 작용해서 [0.9, 0.05, 0.05]와 같이 softmax 함수가 바뀌게 됩니다. 따라서 <b>높은 확률의 단어만 집중적으로 뽑히게 됩니다.</b>

In [0]:
# logits input : [batch_size, vocab_size]
def top_k_logits(logits, k):
  if k == 0:
    return logits
  
  values, _ = tf.math.top_k(logits, k=k) # logits 중 상위 k개를 뽑는다
  min_values = values[:, -1] # 가장 작은 값

  return tf.where(logits < min_values, tf.ones_like(logits, dtype=logits.dtype) * -1e10, logits) # 만약 logits이 가장 작은 값보다 작으면, -1e10 처리한다.

def top_p_logits(logits, p):
  batch = tf.shape(logits)[0]

  sorted_logits = tf.sort(logits, direction='DESCENDING', axis=-1) # logits을 softmax 함수를 적용해서 확률로 만든 후 큰 값이 가장 위로 나오게 sort한다
  cumulative_probs = tf.cumsum(tf.nn.softmax(sorted_logits, axis=-1), axis=-1) # 누적확률분포를 구한다
  indices = tf.stack([tf.range(0, batch), tf.maximum(tf.reduce_sum(tf.cast(cumulative_probs <= p, tf.int32), axis=-1) -1, 0)], axis=-1) 
  # 누적 확률 분포가 임계점 p 를 넘어서기 직전의 인덱스를 구한다
  # indices shape -> [batch_size, 1]

  min_values = tf.gather_nd(sorted_logits, indices) # 배치별 최소 확률
  
  return tf.where(logits < min_values, tf.ones_like(logits) * -1e10, logits) # batch 에서 최소 확률보다 작은 부분은 -1e10을 곱한다.

def sample_sequence(context=None, seq_len=512, temperature = 1, top_k=8, top_p=8, nucleus_sampling=True):
  # context 변수는 string 으로 받는다.

  if context == None:
    print("Feed me some context to generate a text..........")
  
  bos_token = [2] # 영어의 시작 부분 
  eos_token = [3] # 영어의 끝나는 부분

  context = tf.cast(sp.EncodeAsIds(context) + eos_token, dtype=tf.int32)
  # 인풋은 포르투갈 어 + 영어의 시작 토큰임

  context = tf.expand_dims(context, 0)
  # context를 텐서로 바꿔주고, shape을 [1, seq_len]으로 바꿔준다.

  prev = context
  output = context
  past = None
  # 처음에는 입력된 것이 없기 때문에 past에 아무것도 없다.
  # 하지만 입력이 되면 past 변수가 나오고 이거를 순환해서 계속 input으로 활용한다.
  
  for i in range(seq_len):
    logits, past = gpt2(prev, training=False, past=past)
    logits = logits[:, -1, :] / tf.cast(temperature, tf.float32)
    logits = top_k_logits(logits, k=top_k)
    
    if nucleus_sampling:

      logits = top_p_logits(logits, p=top_p)
    samples = tf.random.categorical(logits, num_samples=1, dtype=tf.int32)

    if tf.equal(samples, eos_token):
      break

    output = tf.concat([output, samples], axis=-1)
    prev = samples

  result = tf.squeeze(output, axis=0)
  result = result.numpy().tolist()
  result = sp.DecodeIds(result)
  result = result.replace("<s>", "[Translated to]==>")
  return result

In [52]:
sample_sequence("este é um problema que temos que resolver.", temperature=1.5)

"este é um problema que temos que resolver. . that 's a problem we have to address ."

In [53]:
pt_val[1], eng_val[1]

('estava sempre preocupado em ser apanhado e enviado de volta .',
 'i was always worried about being caught and sent back .')

In [54]:
for i in range(300,400):
  print(sample_sequence(pt_val[i], temperature=0.7, top_k=5), "<<answer>> :", eng_val[i])

vários anos depois do projeto do relógio a energia solar , uma jovem minha conhecida sugeriu que talvez sexo seja tão bom por causa dos cinco sentidos . answer : many years after the solar powered clocks project , a young lady i knew suggested maybe sex is so good because of the five senses .
duas semanas depois , toda a gente estava a tentar entrar na nossa aula . two weeks later , everyone was trying to get in our class . answer : two weeks later , everyone was trying to get into our class .
antes disso , tinha trabalhado em pintura . , i had worked in paint . answer : prior to this time , i had been working as a painter .
todos juntos construímos humanae . we all made up the first together . answer : we all together build humanae .
qual será a cor dos vossos filhos ? who will be the color ? answer : what will be the color of your children ?
eles até usarão ferramentas como o scuinst trojan para infetar o seu computador com um trojan , que lhes permite ver toda sua comunicação ; ouvi